In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import imgaug.augmenters as ia
from sklearn.datasets import make_moons
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
image_directory = "/content/drive/MyDrive/capstone/all_img"
label_directory = "/content/drive/MyDrive/capstone/all_label"

image_files = [f for f in os.listdir(image_directory) if f.endswith((".jpg", ".png", ".jpeg"))]

label_files = [os.path.splitext(f)[0] + ".txt" for f in image_files]

image_paths = [os.path.join(image_directory, f) for f in image_files]
label_paths = [os.path.join(label_directory, f) for f in label_files]


In [ ]:
def parse_label(label_path):
    with open(label_path, 'r') as label_file:
        line = label_file.readline().strip()

    parts = line.split()
    class_id = int(parts[0])
    x_center, y_center, width, height = map(float, parts[1:])

    return class_id


In [ ]:
def create_yolo_model(input_shape, num_predictions=5):
    input_layer = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    output_layer = layers.Dense(num_predictions)(x)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
def prepare_dataset(image_paths, label_paths, input_size):
    images = []
    labels = []
    for image_path, label_path in zip(image_paths, label_paths):
        image = cv2.imread(image_path)
        image = cv2.resize(image, input_size)

        images.append(image)
        labels.append(parse_label(label_path))

    return np.array(images), np.array(labels)

In [ ]:
input_shape = (240, 240, 3)

In [ ]:
images, labels = prepare_dataset(image_paths, label_paths, input_shape[:2])

In [ ]:
print("Number of images:", len(images))
print("Number of labels:", len(labels))

Number of images: 5532
Number of labels: 5532


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
batch_size = 8

num_epochs = 80

In [ ]:
yolo_model = create_yolo_model(input_shape)
yolo_model.summary()
yolo_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 240, 240, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 120, 120, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 60, 60, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856 

In [ ]:
def format_labels(labels):
    formatted_labels = []
    for label in labels:
        class_id = label
        formatted_label = [class_id]
        formatted_labels.append(formatted_label)
    return np.array(formatted_labels)

formatted_labels = format_labels(labels)

In [ ]:
history = yolo_model.fit(
    images, formatted_labels,
    batch_size=batch_size,
    epochs=num_epochs,
)



Epoch 1/80
692/692 [==============================] - 642s 925ms/step - loss: 5.3105 - accuracy: 0.0161
Epoch 2/80
692/692 [==============================] - 641s 926ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 3/80
692/692 [==============================] - 642s 928ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 4/80
692/692 [==============================] - 639s 922ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 5/80
692/692 [==============================] - 634s 916ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 6/80
692/692 [==============================] - 634s 916ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 7/80
692/692 [==============================] - 633s 915ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 8/80
692/692 [==============================] - 635s 918ms/step - loss: 5.3422 - accuracy: 0.0074
Epoch 9/80
328/692 [=============>................] - ETA: 5:32 - loss: 5.4400 - accuracy: 0.0053

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

plt.savefig('es_loss.png')
plt.show()